In [1]:
import torch
from torch import Tensor

from ambitsinn import GaussianTrawlProcess, OUProcess

In [2]:
def ou_process_mean(process: OUProcess) -> float:
    return process.mu.item()


def ou_process_var(process: OUProcess) -> float:
    return (process.sigma**2 / (2 * process.lambda_)).item()


def ou_process_acf(x: Tensor, process: OUProcess) -> Tensor:
    return torch.exp(-process.lambda_ * x)

In [3]:
mu: float = 0.0
sigma: float = 0.5
lambda_: float = 2.0

In [4]:
process = OUProcess(mu, sigma, lambda_)


def process_acf(x: Tensor) -> Tensor:
    return ou_process_acf(x, process)


trawl_representation = GaussianTrawlProcess(
    process_acf,
    ou_process_mean(process),
    ou_process_var(process),
)

In [5]:
x = torch.linspace(-5.0, 5.0, 1000)
print("Differnece in PDF: ", torch.max((process.pdf(x) - trawl_representation.pdf(x)).abs()).item())

times = torch.linspace(0, 10, 1000)
print("Differnece in ACF: ", torch.max((process.acf(times) - trawl_representation.acf(times)).abs()).item())

times = torch.linspace(0, 1, 25)
theta = torch.empty(100, 25).uniform_(-2.0, 2.0)
print(
    "Differnece in cumulant: ",
    torch.max((process.cumulant(theta, times) - trawl_representation.cumulant(theta, times)).abs()).item(),
)

Differnece in PDF:  0.0
Differnece in ACF:  0.0
Differnece in cumulant:  4.76837158203125e-07
